<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/P2_5_3_TGINE_JesusGarciaSalmeron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parte 5.3 - Aplicar el uso de Transformers

In [ ]:
!pip3 install -U transformers datasets torch
!pip3 install accelerate

In [4]:
import transformers
from transformers import AutoModelForSequenceClassification

RoBERTa_spanish = 'dccuchile/albert-base-spanish'

NUM_LABELS = 3

# Cargamos el modelo para clasificación en Pytorch
bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(RoBERTa_spanish, num_labels=NUM_LABELS)
# Cargamos el tokenizer de este modelo
tokenizer = transformers.AutoTokenizer.from_pretrained(RoBERTa_spanish)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/47.8M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at dccuchile/albert-base-spanish and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [5]:
from google.colab import files

uploaded = files.upload()

Saving channelsData.zip to channelsData.zip


In [6]:
import zipfile

zip_file_path = 'channelsData.zip'
extracted_folder_path = 'channelsData/'

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)
    print("Archivo ZIP descomprimido correctamente.")
except zipfile.BadZipFile:
    print("El archivo no es un ZIP válido.")

Archivo ZIP descomprimido correctamente.


In [8]:
import json
import os
from sklearn.model_selection import train_test_split
import pandas as pd

# Train
train_data = []
train_labels = []
# Validation
test_data = []
test_labels = []

# Directorio padre
directory = 'channelsData'

# Contadores para saber el numero de canales
count_cocina = 0
count_deportes = 0
count_viaje = 0
missing = 0

# Recorrer los directorios en channelsData
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            data_type = data['type']
            if data_type == "Cocina":
              count_cocina += 1
              actual_type = count_cocina
            elif data_type == "Deportes":
              count_deportes += 1
              actual_type = count_deportes
            elif data_type == "Viaje":
              count_viaje += 1
              actual_type = count_viaje

            data_videos = data['videos']
            for video in data_videos:
              if actual_type <= 8:
                if video['description'] == "":
                  missing+=1
                train_data.append(video['description'])
                train_labels.append(data['type'])
              else:
                if video['description'] == "":
                  missing+=1
                test_data.append(video['description'])
                test_labels.append(data['type'])



print("FALTAN ->",str(missing))

# Crear DataFrames con nombres de columnas
df_train = pd.DataFrame(data=train_data, columns=['description'])  # Agrega los nombres de columna que desees
df_train['label'] = train_labels  # Agrega la columna de etiquetas al DataFrame de entrenamiento

df_test = pd.DataFrame(data=test_data, columns=['description'])  # Nombres de columnas para el DataFrame de prueba
df_test['label'] = test_labels  # Agrega la columna de etiquetas al DataFrame de prueba

# Definir un diccionario que mapee las etiquetas a números
label_mapping = {"Cocina": 0, "Deportes": 1, "Viaje": 2}

# Aplicar la transformación a las etiquetas en los DataFrames
df_train['label'] = df_train['label'].map(label_mapping)
df_test['label'] = df_test['label'].map(label_mapping)

# Creamos el dataset de validacion
p_eval = 0.2
df_training, df_eval = train_test_split (df_train, test_size = p_eval)

print("Ejemplos usados para entrenar: ", len(df_training))
print("Ejemplos usados para evaluar: ", len(df_eval))
print("Ejemplos usados para test: ", len(df_test))

df_training.head()

FALTAN -> 202
Ejemplos usados para entrenar:  1920
Ejemplos usados para evaluar:  480
Ejemplos usados para test:  600


,description,label
1496,ↆↆↆ APRENDE CONMIGO \n◙ Libro gratis de fotogr...,2
977,"Lesoto, un país dentro de Sudáfrica y escondid...",2
1169,Contrastes es la mejor excursión de Nueva York...,2
1636,Goleada del Barcelona al Real Madrid 0-4 en el...,1
439,👇🏻VER LOS INGREDIENTES👇🏻\n\n⭐No te pierdas mi ...,0


A continuación preparamos los conjuntos de entrenamiento, evaluación y test en **Pytorch** para poder hacer el entrenamiento del modelo.

In [9]:
import torch
import datasets

# Los datasets se preparan dde manera distinta a Tensorflow
class TGINEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments

batch_train_size = 16
batch_val_size = 64

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = './results',
  logging_dir = './logs',
  per_device_train_batch_size = batch_train_size,
  per_device_eval_batch_size = batch_val_size
)


tokenized_train_dataset = tokenizer (df_training.description.tolist (),  truncation=True, padding = True)
tokenized_eval_dataset = tokenizer (df_eval.description.tolist (), truncation=True, padding = True)
tokenized_test_dataset = tokenizer (df_test.description.tolist (), truncation=True, padding = True)


# Como antes, las etiquetas deben ser numéricas para poder entrenar.
# Preparamos los 3 datasets para hacer el finetuning
train_dataset = TGINEDataset (tokenized_train_dataset, df_training.label.tolist())
eval_dataset = TGINEDataset (tokenized_eval_dataset, df_eval.label.tolist())
test_dataset = TGINEDataset (tokenized_test_dataset, df_test.label.tolist())

from transformers import Trainer

trainer = Trainer (
    model = bert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()


print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval ()
print (json.dumps (trainer.evaluate (), indent = 2))


# Salvamos el modelo reentrenado
modelo ='modeloReentrenadoPytorch'
bert_class_model_pytorch.save_pretrained (modelo)
tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))

<ipython-input-9-7ac5641a22b0>:21: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Step,Training Loss


PREDICCIONES SOBRE EVAL


{
  "eval_loss": 0.06428329646587372,
  "eval_accuracy": 0.96875,
  "eval_runtime": 15.8845,
  "eval_samples_per_second": 30.218,
  "eval_steps_per_second": 0.504,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.27724704146385193,
  "test_accuracy": 0.9,
  "test_runtime": 19.6755,
  "test_samples_per_second": 30.495,
  "test_steps_per_second": 0.508
}
